In [3]:
import os
import pandas as pd

def get_gif_paths(directory):
    gif_paths = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.lower().endswith('.gif'):
                gif_paths.append(os.path.join(root, file))
    return gif_paths

def save_paths_to_excel(paths, output_file):
    df = pd.DataFrame(paths, columns=['GIF Paths'])
    df.to_excel(output_file, index=False)


directory = 'exercise_collection'
output_file =  'output.xlsx'
    
gif_paths = get_gif_paths(directory)
save_paths_to_excel(gif_paths, output_file)
    


In [12]:
import pandas as pd
import json
import os

def create_json_from_excel_no_headers(excel_file, json_file):
    # Read the Excel file without headers
    df = pd.read_excel(excel_file, header=None)
    
    # Initialize the list to store JSON objects
    json_list = []
    
    # Iterate through the DataFrame rows
    for idx, row in df.iterrows():
        path = row[0]
        tags = row[1].split(',') if pd.notna(row[1]) else []
        
        # Derive name from path
        file_name = os.path.basename(path)
        name = os.path.splitext(file_name)[0].replace('_', ' ')
        
        # Create JSON object
        json_obj = {
            "id": idx + 1,
            "name": name,
            "path": '../assets/'+path.replace('\\', '/'),
            "tags": tags
        }
        
        # Add JSON object to the list
        json_list.append(json_obj)
    
    # Write the list of JSON objects to a file
    with open(json_file, 'w') as f:
        json.dump(json_list, f, indent=4)

if __name__ == "__main__":
    excel_file = 'Exercise_Data.xlsx'  # The Excel file you created earlier
    json_file = 'Exercise_Data.json'   # The output JSON file
    
    create_json_from_excel_no_headers(excel_file, json_file)
    
    print(f"JSON file has been saved to {json_file}")


JSON file has been saved to Exercise_Data.json


In [13]:

import json
with open('Exercise_Data.json') as f:
  data = json.load(f)

for d in data:

  print('\'' +d['path']+'\'' + ':' + f"require({'\'' +d['path']+'\''}),")

'../assets/exercise_collection/Barbell/Behind_the_Neck_Press.gif':require('../assets/exercise_collection/Barbell/Behind_the_Neck_Press.gif'),
'../assets/exercise_collection/Barbell/Bench_Press.gif':require('../assets/exercise_collection/Barbell/Bench_Press.gif'),
'../assets/exercise_collection/Barbell/Bent_Over_Row.gif':require('../assets/exercise_collection/Barbell/Bent_Over_Row.gif'),
'../assets/exercise_collection/Barbell/Bicep_Curl.gif':require('../assets/exercise_collection/Barbell/Bicep_Curl.gif'),
'../assets/exercise_collection/Barbell/Bulgarian_Split_Squat.gif':require('../assets/exercise_collection/Barbell/Bulgarian_Split_Squat.gif'),
'../assets/exercise_collection/Barbell/Close_Grip_Bench_Press.gif':require('../assets/exercise_collection/Barbell/Close_Grip_Bench_Press.gif'),
'../assets/exercise_collection/Barbell/Deadlift.gif':require('../assets/exercise_collection/Barbell/Deadlift.gif'),
'../assets/exercise_collection/Barbell/Decline_Bench_Press.gif':require('../assets/exerc

In [18]:
from PIL import Image, ImageSequence
import cv2
import numpy as np

def replace_white_with_transparency(input_gif, output_gif):
    # Open the original GIF
    gif = Image.open(input_gif)
    
    frames = []
    for frame in ImageSequence.Iterator(gif):
        # Convert frame to RGB
        frame_rgb = frame.convert("RGB")
        
        # Convert to numpy array
        frame_np = np.array(frame_rgb)
        
        # Convert to BGR (OpenCV uses BGR by default)
        frame_np = frame_np[:, :, ::-1].copy()
        
        # Create a mask where white is detected
        lower_white = np.array([255, 255, 255], dtype=np.uint8)
        upper_white = np.array([255, 255, 255], dtype=np.uint8)
        mask = cv2.inRange(frame_np, lower_white, upper_white)
        
        # Create an alpha channel
        alpha = cv2.bitwise_not(mask)
        
        # Combine the original image with the alpha channel
        frame_bgra = cv2.merge([frame_np, alpha])
        
        # Convert back to PIL Image
        frame_pil = Image.fromarray(frame_bgra, 'RGBA')
        
        frames.append(frame_pil)
    
    # Save the frames as a new GIF
    frames[0].save(output_gif, save_all=True, append_images=frames[1:], loop=0, duration=1000, transparency=0)

# Example usage
input_gif = 'Behind_The_Neck_Press.gif'
output_gif = 'Behind_The_Neck_Press2.gif'
replace_white_with_transparency(input_gif, output_gif)


In [ ]:
import torch
from torchvision import transforms
from PIL import Image
import numpy as np
import os
import cv2

# Define the U^2-Net model
class U2NET(torch.nn.Module):
    def __init__(self, in_ch=3, out_ch=1):
        super(U2NET, self).__init__()
        # Model definition goes here
        # For brevity, I am skipping the detailed implementation
        # You should refer to the official U^2-Net implementation

    def forward(self, x):
        # Forward pass implementation
        # For brevity, I am skipping the detailed implementation
        pass

def load_u2net_model(model_path):
    model = U2NET(3, 1)
    model.load_state_dict(torch.load(model_path, map_location='cpu'))
    model.eval()
    return model

def remove_background_u2net(image, model):
    preprocess = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((320, 320)),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
    ])
    input_tensor = preprocess(image).unsqueeze(0)
    with torch.no_grad():
        d1, _, _, _, _, _, _ = model(input_tensor)
        mask = d1[:, 0, :, :]
        mask = (mask > 0.5).cpu().numpy().astype(np.uint8)
        mask = cv2.resize(mask[0], (image.shape[1], image.shape[0]))
    return mask

def process_gif_with_u2net(gif_path, output_path, model):
    gif = cv2.VideoCapture(gif_path)
    frames = []
    success, frame = gif.read()
    while success:
        mask = remove_background_u2net(frame, model)
        frame[mask == 0] = [0, 0, 0, 0]
        frames.append(frame)
        success, frame = gif.read()

    height, width, _ = frames[0].shape
    fourcc = cv2.VideoWriter_fourcc(*'GIF')
    out = cv2.VideoWriter(output_path, fourcc, gif.get(cv2.CAP_PROP_FPS), (width, height))

    for frame in frames:
        out.write(frame)

    out.release()
    gif.release()

model_path = 'u2net.pth'  # Path to the downloaded U^2-Net model weights
model = load_u2net_model(model_path)

input_folder = 'path/to/your/gifs'
output_folder = 'path/to/save/processed/gifs'

for filename in os.listdir(input_folder):
    if filename.endswith('.gif'):
        process_gif_with_u2net(os.path.join(input_folder, filename), os.path.join(output_folder, filename), model)
